# A Neural Network Based Approach to Medical Image Recognition
### Jack Savage, Kyle Burack, Tyler Seppala, Josh Michelberg

---
**Introduction**:

Many communities around the world have little to no access to modern health care. Within our own country, there are two major groups with limited access: those without money for health-care, and those who are far from their nearest healthcare provider. In both circumstances listed, artificial intelligence provides a solution. In poverty-stricken areas, human doctors are costly while an open-source software-based diagnosis system would be cheap. In rural areas without access to experienced or well-trained doctors, medical technicians would be able to perform at a similar level.

![alt text](doctor_shortages.jpg "Doctor Shortages")
*Relevent graphic showcasing shortages of doctors within the United States*

_Further information on the 'HPSA score' metric used in the image can be found [here](https://bhw.hrsa.gov/shortage-designation/hpsas)_

- **Talk about previous approaches to medical image recognition (CITE STUFF HERE) (JOSH)**

## Past Work:

We began this project last semester with no knowledge of neural networks. ~75% of our work last semester was in researching our topic while ~25% was becoming familiar with the high-level API [Keras](https://keras.io/). This semester, we've been able to focus completely on model development and assessment and have created 2 unsuccessful models and 1 moderately successful model. 

Our initial model was of the Xception architecture **(XCEPTION CITATION)** and was trained on the untouched dataset divided into subclasses. Xception is an adjusted version of Google's publicly released Inception model **(INCEPTION CITATION)**. Both architectures contain specialized layers where multiple convolutional operations are performed on the same tensor. Inception modules concatonate the outputs of these operations while Xception maps the all outputs directly to a tensor of the same x and y dimensions. 


<img src="inception_module.JPG" alt="inception_module" style="width: 400px;"/>
<img src="xception_module.JPEG" alt="xception_module" style="width: 400px;"/>

We chose Xception because it's been reliably proven to be more accurate than the other architecture while also being relatively quick to train ([compared to the other architectures available natively within keras](https://keras.io/applications/)). This model proved to be a failure due to reasons discussed in the **Dataset** section. This model was extremely biased towards classifying images as being heathy. 

As the use of single stage algorithms wasn’t proving successful, we decided to try a two stage object detection network called RetinaNet. The backbone of this model is an algorithm known as a feature pyramid network (FPN). The feature pyramid takes an input image and breaks it down to a range of different resolutions, forming a feature map out of them (Lin et al. 2017). The feature map at each resolution, or layer, is then fed in succession through two fully convolutional neural networks (FCN’s). The first stage of object detection, carried out by the first FCN,  is object classification. The algorithm breaks the image from the given FPN level into candidate regions, enclosing each one with an anchor box. These candidate regions are parts of the image that the network, based on its training, suspect contain pneumonia. This new image, which contains the anchor boxes, is then fed into the second FCN for the second stage: box regression. This FCN takes a closer, more precise look at the enclosed regions and attempts to resize the anchor boxes to closely fit determined cases of pneumonia. If it is determined that there is no pneumonia within the anchor box, the box will be deleted entirely. The output of each layer is then compared, and the information is accumulated to produce one output image. Ideally, this image will be blank if healthy, or have anchor boxes tightly fit around any cases of pneumonia (Lin et al. 2017).

Our hope in this new approach was that we could improve training speed and classification accuracy by avoiding class imbalance. Class imbalance in this case describes a situation where the classes the network is trying to identify are scarce within each training image, and there is a lot of empty space void of useful information (Lin et al. 2017). We found in our previous attempts that using one network to closely analyze entire images of lungs wasted a lot of training time, as many pneumonia cases exist only in scarce regions. Close analysis of the vast, healthy parts of the lungs is not necessary. We believed that RetinaNet would solve this problem, as it throws away most of the unnecessary information in the first stage. 

As we are fairly new to programming neural networks, and RetinaNet is an existing model that has been successful on other datasets, we decided to utilize a premade model and alter the subnetworks. Here is the repository from which we got the initial code: https://github.com/fizyr/keras-retinanet. We changed the image classification subnetwork to a preset model called ResNet50, which we hoped would better suit our type of training data. Ultimately, the model was not successful in identifying pneumonia, returning an accuracy below 0.01. On close analysis of our model, we found that the issue was is the classification stage. When evaluated on testing data, anchor boxes were never produced at all, meaning that the classification process was not detailed enough to identify any candidate regions, and the regression network never got an input. 


Unfortunately, our team lost the hard drive containing training and validation metrics for our first two models so will not be able to discuss these models' results in more detail.

## This Semester's Work

**Intro:**

This semester
- Dataset **(BURACK)** https://www.kaggle.com/c/rsna-pneumonia-detection-challenge?login=true
    - Discuss dimensionality issues
    - Discuss discrepencies in classes
- Preprocessing of data **(JACK)**
    - Discuss techniques such as over/under-sampling

**Our network**

**Results**:
- Tensorboard graphs (BOTH MODELS) **(JACK)**
    - Interpretations
- Evaluate network on test imaging **(JACK)**
    - Discuss how discrepencies in dataset could account for results

#### Xception training accuracy
![alt_text](training_accuracy_xception_v2.PNG)
#### Xception training loss
![alt_text](training_loss_xception_v2.PNG)

**Improvements for Next Semester:**

While this semester's results aren't ideal, we've built a robust codebase to build upon. In less than a year, our group has gone from not knowing anything about neural networks to developing and evaluating multiple models rapidly. In the spring, we have many ways improve our model's accuracy. First, we could train more CNNs on our dataset and use ensemble them with our existing model (output is average of each networks output). This is a technique thats been used to great effect elsewhere **ENSEMBLE CITATION**. As we did with RetinaNet, we can implement new algorithms discovered in literature if traditional methods fail. We could augment our dataset by randomly altering training images, effectively creating more data for our network to learn from. Finally, we could attempt to improve our existing model by tweaking hyperparameters such as number of epochs and steps per epoch which will give the model more training iterations over the whole dataset. This final option likely leads to overfitting, as we can see that our loss and accuracy have already sufficiently flattened out. If the slope was more drastic in the final epochs, this would suggest that our model's loss hasn't hit a local minimum and more training is required. 

**COMPARE POSITIVE PREDICTIVE VALUES**

- Ensembling models
- Evaluate new methods from literature
- Evaluate current infrastructure on new dataset
- Train more models

**Conclusion:**

In conclusion, our

## Code Snippits to Write

In [1]:
import cv2
import keras
import matplotlib
import pandas as pd
import numpy as np
import keras.layers as layers
from keras.models import Model
from keras.models import load_model
from keras.layers import Average, Input
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
xception_final = load_model('xception15.hdf5')

### Evaluating network on all test data

In [5]:
data_dir = r'C:\Users\jecks\Documents\School\GCI\gci_data\test'

test_datagen = ImageDataGenerator()
test_datagen = test_datagen.flow_from_directory(data_dir,
                                                batch_size = 1,
                                                target_size=(1024,1024),
                                                class_mode='categorical')

Found 208 images belonging to 2 classes.


In [7]:
test_history = xception_final.evaluate_generator(test_datagen, verbose=1)

208/208 [==============================] - 2609s 13s/step


In [11]:
print(test_history)
print(xception_final.metrics_names)

[0.6660128929114972, 0.6394230769230769]
['loss', 'acc']


### Evaluating positive predictive value

In [ ]:
# only 1 pneumonia negative image used in this data generator
positive_data_dir = r'C:\Users\jecks\Documents\School\GCI\gci_data\positive_test'

test_datagen = ImageDataGenerator()
test_datagen = test_datagen.flow_from_directory(positive_data_dir,
                                                batch_size = 1,
                                                target_size=(1024,1024),
                                                class_mode='categorical')

In [ ]:
positive_test_history = xception_final.evaluate_generator(test_datagen, verbose=1)

 29/105 [=======>......................] - ETA: 15:55

In [ ]:
print(positive_test_history)
print(xception_final.metrics_names)

## Resources
- https://www.ruralhealthinfo.org/topics/healthcare-access

Chollet F. 2017. Xception: Deep Learning with Depthwise Separable Convolutions. 2017 IEEE Conference on Computer Vision and Pattern Recognition (CVPR).

http://papers.nips.cc/paper/1001-neural-network-ensembles-cross-validation-and-active-learning.pdf